In [56]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re
import itertools as it

In [1]:
#These below functions sift through the candidate header for information
def name_extractor(candidate):
            return re.findall(r'[A-Z]\w+ [A-Z]\w+',candidate)[0]
def party_extractor(candidate):
            return re.findall(r'\(\w\)',candidate)[0]
def incumbent_finder(candidate):
            incumbent = re.findall(r'Incumbent',candidate)
            if len(incumbent)>0:
                return incumbent[0]
            else: 
                return 'N/a'
def winner_finder(candidate):
            winner = re.findall(r'Winner',candidate)
            if len(winner)>0:
                return winner[0]
            else:
                return 'N/a'
def percentage_vote_finder(candidate):
            return re.findall(r'(?<=\()\d+.\d\%',candidate)
#This will be based on the website you are on, there are different websites for each State,District combo. That's why soup is input
def state_and_district_finder(soup):
            string = str(soup.findAll('h1'))
            state = re.findall(r'(?<=>)[A-Z]\w+', string)
            district = re.findall(r'District \d{2}', string)
            return state, district; 

In [57]:
#read in State and District info to pull each state and district url with candidates
state_info = pd.read_csv('../webscraping_open_secrets-malted_milk_balls/Data/state_abr_districts.csv')

In [82]:
#create lists of state abbreviations and district numbers
abb = state_info['Abbreviation'].unique().tolist()
dist = state_info['District Numbers'].astype(str).str.zfill(2).unique().tolist()

In [86]:
url = [f'https://www.opensecrets.org/races/candidates?cycle=2020&id={state}{district}&spec=N' for state, district in it.product(abb, dist)]
url

['https://www.opensecrets.org/races/candidates?cycle=2020&id=AL01&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL02&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL03&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL04&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL05&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL06&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL07&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL08&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL09&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL10&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL11&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL12&spec=N',
 'https://www.opensecrets.org/races/candidates?cycle=2020&id=AL13&spec=N',
 'https://www.opensecrets